<a href="https://colab.research.google.com/github/achanhon/AdversarialModel/blob/master/vitesse_tirage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import torch
import time

In [2]:
p = torch.rand(4)*4-2
MAX,a = p.max(0)
Mean = p.mean()
target = 0.9*MAX+0.1*Mean
print(p,MAX,Mean,target)

tensor([ 1.0868,  1.9435, -0.3685, -0.2120]) tensor(1.9435) tensor(0.6125) tensor(1.8104)


In [3]:
T0 = time.time()
out = []
for _ in range(100000):
  if random.random()<0.9:
    out.append(int(a))
  else:
    out.append(int(random.random()*4))
T = time.time()
print(T-T0)
E = [p[i] for i in out]
E = sum(E)/len(E)
print(E)

0.08209848403930664
tensor(1.8078)


In [4]:
soft = torch.nn.functional.softmax(p,dim=0)
softmean = (p*soft).sum()
print(soft,softmean)
assert softmean<target

# seuil * MAX + (1-seuil)*softmean = target
seuil = (target-softmean)/(MAX-softmean)

T0 = time.time()
out = []
for _ in range(100000):
  if random.random()<seuil:
    out.append(int(a))
  else:
    out.append(int(torch.multinomial(soft, 1).item()))
T = time.time()
print(T-T0)
E = [p[i] for i in out]
E = sum(E)/len(E)
print(E)

tensor([0.2590, 0.6099, 0.0604, 0.0707]) tensor(1.4296)
2.0684547424316406
tensor(1.8088)
